In [1]:
import matplotlib.pyplot as plt
from matplotlib  import rc
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import plotly.express as px
from plotly import graph_objs as go


In [2]:
bank_tract = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [3]:
grouped = bank_tract.groupby(['census_tract', 'which_bank']) 
denial_rates = grouped['denial_rate'].mean()

bank_of_west = denial_rates.loc[(slice(None), 'Bank of West')]
all_other_banks = denial_rates.loc[(slice(None), 'All Other Banks')]
percent_difference = bank_of_west - all_other_banks

df_percent_difference = pd.DataFrame({'census_tract': percent_difference.index.get_level_values(0), 'percent_difference': percent_difference.values})

bank_tract = pd.merge(bank_tract, df_percent_difference, on='census_tract', how='left')


In [4]:
shape_az = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_04_tract.zip") #.to_crs(epsg=32617)
shape_ca = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_06_tract.zip") #.to_crs(epsg=32617)
# shape_in = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_18_tract.zip") #.to_crs(epsg=32617)

shape_all = pd.concat([shape_az, shape_ca], ignore_index=True)
shape_all['census_tract'] = shape_all['GEOID'].astype(np.int64)

In [5]:
tract_with_shape = shape_all.merge(bank_tract, how = 'inner', on = ['census_tract'],
                                   indicator = True, validate= '1:m') 

In [6]:
bank_tract ['white_rate']        = (bank_tract ['WhitePop']) / bank_tract ['Tot.Pop'] * 100
bank_tract ['majority-minority'] = bank_tract ['white_rate'] < 50

bank_tract = bank_tract.query('(state == 4 & county in [13,19]) | (state == 6 & county in [37,75, 81, 1,85])')
 # | (state == 18 & county in [97])

In [7]:
# concerns = tract_with_shape.query("`majority-minority`").copy()
# # we need this for the "area of concern" red marks
# concerns['lon'] = concerns['INTPTLON'].astype(float)
# concerns['lat'] = concerns['INTPTLAT'].astype(float)

In [8]:
listofshading = ['percent_difference']

In [9]:
import plotly.colors as colors
colorscale = colors.sequential.Greens[::-1] + colors.sequential.Reds

for v in listofshading:

    fig = px.choropleth_mapbox(tract_with_shape.set_index('GEOID'),
                               geojson=tract_with_shape.geometry,
                               locations=tract_with_shape.index,
                               opacity=.7,
                               color=v, color_continuous_scale=colorscale,
                               center={"lat": 33.44, "lon": -112.074036},
                               mapbox_style="open-street-map",
                               zoom=5)

    # fig.add_scattermapbox(
    #     lat = concerns['lat'],
    #     lon = concerns['lon'],
    #     mode = 'markers',
    #     marker_size=12,
    #     marker_color='red'
    # )

    fig.update_layout(
        title=v,
        autosize=False,
        width=1000,
        height=1000,
        margin={"r":0,"t":0,"l":0,"b":0}
    )
    fig.write_html(f"../images/plotly/{v}.html")

In [10]:
#